In [1]:
import pandas as pd

In [6]:
data = pd.read_csv('/Users/ShresthaRupesh/Desktop/dataset_for_first_decision_tree.csv')

In [55]:
partitionedData = pd.DataFrame()
partitionedData = data[data.age == 'youth']
len(partitionedData.index)



5

In [52]:
classes = list(set(data.Class))
from collections import Counter
Class = data.Class
counts = Counter(dict(Class).values())
sum(counts.values())

age              5
income           5
student          5
credit_rating    5
Class            5
dtype: int64


In [153]:
from collections import Counter
import numpy as np

class Node:
    def __init__(self):
        self.attribute = ""
        self.children = []
        self.branches = []
        self.isLeaf = False
        self.ClassLabel = ""

class DecisionTree:
    
    def __init__(self,train_data):
        self.train_data = train_data
        self.size = len(self.train_data.index)
        self.attributes = list(self.train_data.columns)
        self.Class = self.attributes[-1]
        self.totalEntropy = self.getEntropy(self.train_data)
        self.root = None
    
    
    def getEntropy(self,data):
        Class = self.Class
        
        classes = list(set(data[Class]))
        counts = Counter(dict(data[Class]).values())
        total = sum(counts.values())
        entropy = 0
        for count in counts.values():
            probability = count/total
            entropy+= -probability*np.log2(probability)
        return entropy
    
    def getAttributeValues(self,attribute):
        return list(set(self.train_data[attribute]))
    
    def partition(self,data,attribute):
        attributeValues = self.getAttributeValues(attribute)
        partitions = []
        for value in attributeValues:
            partitions.append(pd.DataFrame(data[data[attribute] == value]).drop(columns = [attribute]))
        return partitions
    
    def getInformationContent(self,data,attribute):
        Class = self.Class
        partitions = self.partition(data,attribute)
        informationContent = 0
        
        for partition in partitions:
            informationContent+= (len(partition.index)/self.size)*self.getEntropy(partition)
        return (informationContent)
    
    def getGain(self,data,attribute):
        return self.totalEntropy - self.getInformationContent(data,attribute)
    
    
    
    def chooseNode(self,data,attribute_list):
       
        args = []
        for i in range(len(self.attributes)-1):
            if self.attributes[i] in attribute_list:
                args.append(self.getGain(data,self.attributes[i]))
            else:
                args.append(0)

        return self.attributes[np.argmax(args)]
    
    def isLeafNode(self,data):
        return len(set(data[self.Class])) == 1
    
    def getClassLabel(self,data):
        return list(set(data[self.Class]))[0]
    
    def constructSubtree(self,data,attribute_list):
        child = Node()
        #print("child",child.attribute)
        child.attribute = self.chooseNode(data,attribute_list)
        #print("child",child.attribute)
        
        child.branches = self.getAttributeValues(child.attribute)
        child.isLeaf = self.isLeafNode(data)
        partitions = self.partition(data,child.attribute)
        #print("partitions",partitions)
        if child.isLeaf:
            child.ClassLabel = self.getClassLabel(data)
           # print("label",child.ClassLabel)
            
        else:
            for partition in partitions:
                attribute_listN = attribute_list[:]
                attribute_listN.remove(child.attribute)
                child.children.append(self.constructSubtree(partition,attribute_listN))
        return child
                
            
    def constructTree(self):
        self.root = Node()
        self.root.attribute = self.chooseNode(self.train_data,self.attributes)
        self.root.branches = self.getAttributeValues(self.root.attribute)
        self.root.isLeaf = self.isLeafNode(self.train_data)
        partitions = self.partition(self.train_data,self.root.attribute)
        if self.root.isLeaf:
            self.root.ClassLabel = self.getClassLabel(self.train_data)
        else:
            for partition in partitions:
                attribute_list = self.attributes[:]
                attribute_list.remove(self.root.attribute)
                self.root.children.append(self.constructSubtree(partition,attribute_list))
    def parseTree(self,node):
        if node.children != []:
            for i in range(len(node.children)):
                child = node.children[i]
                print(node.branches[i])
                if child.isLeaf:
                    print(child.ClassLabel)
                else:
                    print(child.attribute)
                self.parseTree(child)
    def parseTreeHelper(self):
        print(20*' /'+self.root.attribute+20*'/')
        #print(self.root.attribute)
        self.parseTree(self.root)
        
        
                
            
if __name__ == '__main__':
    classifier = DecisionTree(data)
    attributeList = list(data.columns)
    classifier.constructTree()
    classifier.parseTreeHelper()
        
        
        
        
        
    
    
    
    
    
    
    
    
    
    

 / / / / / / / / / / / / / / / / / / / /age////////////////////
youth
student
yes
yes
no
no
middle_aged
yes
senior
credit_rating
excellent
no
fair
yes
